# 1: MTC MNL Constrained Mode Choice

In [ ]:
import pandas as pd

import larch as lx
from larch import P, X

In [ ]:
# TEST
pd.set_option("display.max_columns", 999)
pd.set_option("expand_frame_repr", False)
pd.set_option("display.precision", 3)
from pytest import approx

This example is a mode choice model built using the MTC example dataset.
First we create the Dataset and Model objects:

In [ ]:
m = lx.example(1)
m.title = "MTC Example 30 (Constrained Simple MNL)"
m.compute_engine = "numba"

Suppose we want to ensure that the implied value of time must be at least $5 per hour.
Our data expresses cost in cents and time in minutes. Given our simple utility formulation,
the target value of time is achieved if the ratio of the cost parameter to the time
parameter exceeds 3.0.

We could explicitly set the ratio at exactly 3.0 by writing the model with only one parameter.

In [ ]:
m_explicit = m.copy()

In [ ]:
m_explicit.utility_ca = P.tottime * X.tottime + P.tottime * 3 * X.totcost
m_explicit.remove_unused_parameters()

In [ ]:
result_explicit = m_explicit.maximize_loglike(stderr=True)

In [ ]:
# TEST
assert dict(result_explicit.x) == approx(
    {
        "ASC_BIKE": -3.160940102309336,
        "ASC_SR2": -2.455774449621098,
        "ASC_SR3P": -4.080513763962432,
        "ASC_TRAN": -1.893503726851137,
        "ASC_WALK": -1.9875386692858579,
        "hhinc#2": -0.0019850591646016313,
        "hhinc#3": 0.0004627579363727124,
        "hhinc#4": -0.00616068731356304,
        "hhinc#5": -0.014087710498599073,
        "hhinc#6": -0.00938519135930957,
        "tottime": -0.001739013257543946,
    }
)
assert result_explicit.logloss == approx(0.7533452255366115)
assert result_explicit.loglike == approx(-3788.5731392236194)
assert result_explicit.message == "Optimization terminated successfully"

In [ ]:
from larch.model.constraints import RatioBound

m.pmaximum = {"totcost": 0, "tottime": 0}

m.constraints = [
    RatioBound("totcost", "tottime", min_ratio=3.0, max_ratio=999.0, scale=100),
]

Having created this model, we can then estimate it:

In [ ]:
# TEST
assert dict(m.required_data()) == {
    "ca": ["totcost", "tottime"],
    "co": ["hhinc"],
    "choice_ca": "chose",
    "avail_ca": "avail",
}
assert m.loglike() == approx(-7309.600971749634)

In [ ]:
result = m.maximize_loglike(stderr=True)

In [ ]:
result

In [ ]:
# TEST
assert result.loglike == approx(-3788.573358)
assert result.logloss == approx(0.753345269140234)
assert result.message == "Optimization terminated successfully"
assert m.total_weight() == 5029.0

In [ ]:
m.parameter_summary()

In [ ]:
# TEST
summary = m.parameter_summary()
# assert_same_text(
#     summary.data.to_markdown(),
#     '''
# | Parameter   |     Value |   Std Err |   t Stat | Signif   |   Null Value | Constrained             |
# |:------------|----------:|----------:|---------:|:---------|-------------:|:------------------------|
# | ASC_BIKE    | -3.16     |  0.309    |   -10.23 | ***      |            0 |                         |
# | ASC_SR2     | -2.46     |  0.103    |   -23.94 | ***      |            0 |                         |
# | ASC_SR3P    | -4.08     |  0.175    |   -23.26 | ***      |            0 |                         |
# | ASC_TRAN    | -1.89     |  0.112    |   -16.87 | ***      |            0 |                         |
# | ASC_WALK    | -1.99     |  0.169    |   -11.77 | ***      |            0 |                         |
# | hhinc#2     | -0.00199  |  0.00154  |    -1.3  |          |            0 |                         |
# | hhinc#3     |  0.000462 |  0.00252  |     0.18 |          |            0 |                         |
# | hhinc#4     | -0.00616  |  0.0018   |    -3.42 | ***      |            0 |                         |
# | hhinc#5     | -0.0141   |  0.0055   |    -2.57 | *        |            0 |                         |
# | hhinc#6     | -0.00941  |  0.00306  |    -3.08 | **       |            0 |                         |
# | totcost     | -0.00522  |  0.000243 |   -21.5  | ***      |            0 | totcost / tottime ≥ 3.0 |
# | tottime     | -0.00174  |  8.09e-05 |   -21.5  | ***      |            0 | totcost / tottime ≥ 3.0 |
#     '''
# )

It is a little tough to read this report because the parameters can show up 
in pretty much any order, as they are not sorted
when they are automatically discovered by Larch.
We can use the reorder method to fix this:

In [ ]:
m.ordering = (
    (
        "LOS",
        "totcost",
        "tottime",
    ),
    (
        "ASCs",
        "ASC.*",
    ),
    (
        "Income",
        "hhinc.*",
    ),
)

In [ ]:
m.parameter_summary()

In [ ]:
# TEST
summary2 = m.parameter_summary()
# assert_same_text(
#     summary2.data.to_markdown(),
#     '''
# |                       |     Value |   Std Err |   t Stat | Signif   |   Null Value | Constrained             |
# |:----------------------|----------:|----------:|---------:|:---------|-------------:|:------------------------|
# | ('LOS', 'totcost')    | -0.00522  |  0.000243 |   -21.5  | ***      |            0 | totcost / tottime ≥ 3.0 |
# | ('LOS', 'tottime')    | -0.00174  |  8.09e-05 |   -21.5  | ***      |            0 | totcost / tottime ≥ 3.0 |
# | ('ASCs', 'ASC_BIKE')  | -3.16     |  0.309    |   -10.23 | ***      |            0 |                         |
# | ('ASCs', 'ASC_SR2')   | -2.46     |  0.103    |   -23.94 | ***      |            0 |                         |
# | ('ASCs', 'ASC_SR3P')  | -4.08     |  0.175    |   -23.26 | ***      |            0 |                         |
# | ('ASCs', 'ASC_TRAN')  | -1.89     |  0.112    |   -16.87 | ***      |            0 |                         |
# | ('ASCs', 'ASC_WALK')  | -1.99     |  0.169    |   -11.77 | ***      |            0 |                         |
# | ('Income', 'hhinc#2') | -0.00199  |  0.00154  |    -1.3  |          |            0 |                         |
# | ('Income', 'hhinc#3') |  0.000462 |  0.00252  |     0.18 |          |            0 |                         |
# | ('Income', 'hhinc#4') | -0.00616  |  0.0018   |    -3.42 | ***      |            0 |                         |
# | ('Income', 'hhinc#5') | -0.0141   |  0.0055   |    -2.57 | *        |            0 |                         |
# | ('Income', 'hhinc#6') | -0.00941  |  0.00306  |    -3.08 | **       |            0 |                         |
#     '''
# )